In [2]:
from sklearn.ensemble import GradientBoostingClassifier
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np

In [13]:
df_train = pd.read_csv('train.csv')
df_test= pd.read_csv('test.csv')

In [14]:
mapping = {"Yes":1,"No":0,"Positive":1,"Negative":0,"Male":0,"Female":1}

In [15]:
X1 = df_train.replace(mapping).drop(columns='ID')
X2 = df_test.replace(mapping).drop(columns='ID')

C:\Users\peaks\AppData\Local\Temp\ipykernel_7520\1381683691.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X1 = df_train.replace(mapping).drop(columns='ID')
C:\Users\peaks\AppData\Local\Temp\ipykernel_7520\1381683691.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X2 = df_test.replace(mapping).drop(columns='ID')


In [10]:
X1

,age,gender,polyuria,polydipsia,sudden_weight_loss,weakness,polyphagia,genital_thrush,visual_blurring,itching,irritability,delayed_healing,partial_paresis,muscle_stiffness,alopecia,obesity,class
0,41,0,1,0,0,1,1,0,0,1,0,1,0,1,1,0,1
1,45,0,0,0,1,1,1,1,0,1,0,1,0,0,0,0,1
2,60,0,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1
3,66,0,1,1,1,1,0,0,1,1,1,0,1,1,0,0,1
4,67,0,1,1,0,1,1,1,0,1,1,0,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,62,1,1,1,1,1,0,0,1,0,0,0,1,0,0,1,1
360,39,1,1,1,1,0,1,0,0,1,0,1,1,0,0,0,1
361,48,1,1,1,1,1,1,0,0,1,1,1,1,0,0,0,1
362,58,1,1,1,1,1,1,0,1,0,0,0,1,1,0,1,1


In [25]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold,cross_validate
from sklearn.model_selection import cross_val_score

In [27]:
import optuna
from optuna.samplers import TPESampler
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice

c:\Users\peaks\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
y = X1['class']
X1 = X1.drop(columns='class')

In [24]:
X_train, X_valid, y_train, y_valid = train_test_split(X1,y, test_size=0.2, random_state=43)

In [31]:
def objective(trial, X, y, cv, scoring):
  """
  An objective function to tune hyperparameters of Gradient Boosting Classifier.
  Args:
    trial: an Optuna trial
    X: DataFrame object, features
    y: Series object, Labels
    cv: k folds to cross-validate
    scoring: String, evaluation metric
  Return:
    Mean test accuracy
  """

  params = {
    "n_estimators": trial.suggest_int("n_estimators", 100, 5000, step = 100),
    "learning_rate": trial.suggest_float("learning_rate", 1e-4, 0.3, log = True),
    "max_depth": trial.suggest_int("max_depth", 3, 9),
    "subsample": trial.suggest_float("subsample", 0.5, 0.9, step = 0.1),
    "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2"]),
    "random_state": 42,
    }
  # Perform cross validation
  gb_class = GradientBoostingClassifier(**params)

  # Compute scores
  scores = cross_validate(gb_class, X, y, cv = cv, scoring = scoring, n_jobs = -1)
  accuracy = scores["test_score"].mean()

  return accuracy

In [32]:
study = optuna.create_study(direction = "maximize")

kf = KFold(n_splits = 5, shuffle = True, random_state = 42)

func = lambda trial: objective(trial, X1, y, cv = kf, scoring = "accuracy")

[I 2024-01-28 02:09:39,067] A new study created in memory with name: no-name-42fb07d5-56d2-4255-83c5-12f0a40d21a8


In [33]:

# Start optimizing with 100 trials
study.optimize(func, n_trials = 100)

[I 2024-01-28 02:09:44,006] Trial 0 finished with value: 0.95882800608828 and parameters: {'n_estimators': 4300, 'learning_rate': 0.024314374102978587, 'max_depth': 6, 'subsample': 0.8, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.95882800608828.
[I 2024-01-28 02:09:46,007] Trial 1 finished with value: 0.9560502283105021 and parameters: {'n_estimators': 700, 'learning_rate': 0.02739358004596193, 'max_depth': 7, 'subsample': 0.6, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.95882800608828.
[I 2024-01-28 02:09:49,675] Trial 2 finished with value: 0.9505327245053271 and parameters: {'n_estimators': 2500, 'learning_rate': 0.02702488428508831, 'max_depth': 8, 'subsample': 0.6, 'max_features': 'log2'}. Best is trial 0 with value: 0.95882800608828.
[I 2024-01-28 02:09:51,013] Trial 3 finished with value: 0.9532724505327245 and parameters: {'n_estimators': 2100, 'learning_rate': 0.15031271666435797, 'max_depth': 8, 'subsample': 0.8, 'max_features': 'sqrt'}. Best is trial 0 

In [34]:
parameters = {'n_estimators': 900, 'learning_rate': 0.0014434856189557883, 'max_depth': 5, 'subsample': 0.8, 'max_features': 'log2'}
#. Best is trial 11 with value: 0.964345509893455.

In [35]:
modelG= GradientBoostingClassifier(**parameters)
modelG.fit(X1, y)

GradientBoostingClassifier(learning_rate=0.0014434856189557883, max_depth=5,
                           max_features='log2', n_estimators=900,
                           subsample=0.8)

In [36]:
kf = KFold(n_splits=20, shuffle=True, random_state=42)
cv_results = cross_val_score(modelG, X1, y, cv=kf)
print("Accuracy for each fold:", cv_results)
print("Average accuracy across all folds:", cv_results.mean())

Accuracy for each fold: [0.89473684 0.94736842 1.         0.94736842 0.94444444 1.
 1.         0.94444444 1.         0.94444444 1.         1.
 0.88888889 1.         1.         1.         0.94444444 1.
 1.         1.        ]
Average accuracy across all folds: 0.9728070175438596


In [37]:
pred=modelG.predict_proba(X2)

In [40]:
pred.round().shape

(156, 2)

In [41]:
pred=pred.T[1]

In [49]:
pred

array([0.89903988, 0.89886674, 0.21897359, 0.89981704, 0.20752039,
       0.23522914, 0.24884074, 0.19595554, 0.22639654, 0.88814006,
       0.8912858 , 0.20924153, 0.90072028, 0.27544695, 0.89020147,
       0.87967185, 0.88200535, 0.8718511 , 0.22858696, 0.90095957,
       0.70714158, 0.83955069, 0.26907844, 0.2186067 , 0.78900202,
       0.2186067 , 0.8718511 , 0.24198421, 0.82690355, 0.901127  ,
       0.89761085, 0.40693262, 0.32032443, 0.90057725, 0.36877577,
       0.901127  , 0.24884074, 0.22490364, 0.89775707, 0.89950584,
       0.90094089, 0.26213312, 0.22858696, 0.64087709, 0.89932685,
       0.8869746 , 0.50987255, 0.89386099, 0.25101237, 0.88824384,
       0.88714203, 0.31780916, 0.78900202, 0.38937084, 0.22422398,
       0.90109167, 0.45074856, 0.23952869, 0.26876162, 0.2186067 ,
       0.89502816, 0.87988444, 0.21572385, 0.83627944, 0.79428652,
       0.22422398, 0.70309455, 0.87967185, 0.90061007, 0.901127  ,
       0.24144345, 0.88642992, 0.66369605, 0.22422398, 0.28779

In [43]:
result = pred.round()

In [44]:
R1 = pd.DataFrame(df_test['ID'])
result_nn1 = pd.DataFrame(result)

In [45]:
mapping2 = {1:'Positive',0:'Negative'}
resultf = result_nn1.replace(mapping2)

In [46]:
R1['class'] = resultf

In [47]:
R1

,ID,class
0,0,Positive
1,1,Positive
2,2,Negative
3,3,Positive
4,4,Negative
...,...,...
151,151,Negative
152,152,Negative
153,153,Negative
154,154,Positive


In [48]:
R1.to_csv('submission4.csv', index=False)
R1.to_csv(r'C:\Users\peaks\Desktop\Submission\submission4.csv', index=False)